# Курсовик по предмету "Рекомендательные системы" Гладышев В.В.

### EDA, фильтрация, feature engineering:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/coursework/cw_eda_glvv.ipynb

### библиотека с функциями метрик:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/metrics.py

### подготовка данных: 

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/utils.py

### класс с моделями:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/recommenders.py

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data          = pd.read_csv(os.path.join(os.pardir, "data", "data.csv"))
user_features = pd.read_csv(os.path.join(os.pardir, "data", "user_features.csv"))
item_features = pd.read_csv(os.path.join(os.pardir, "data", "item_features.csv"))

# схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
recommender = MainRecommender(data_train_lvl_1, item_features, user_features)